In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pandas as pd
import numpy as np

In [11]:
from sklearn.datasets import fetch_openml
data=fetch_openml(name='bank-marketing',version=1,as_frame=True)
df=data.frame
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Class
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   V1      45211 non-null  int64   
 1   V2      45211 non-null  category
 2   V3      45211 non-null  category
 3   V4      45211 non-null  category
 4   V5      45211 non-null  category
 5   V6      45211 non-null  int64   
 6   V7      45211 non-null  category
 7   V8      45211 non-null  category
 8   V9      45211 non-null  category
 9   V10     45211 non-null  int64   
 10  V11     45211 non-null  category
 11  V12     45211 non-null  int64   
 12  V13     45211 non-null  int64   
 13  V14     45211 non-null  int64   
 14  V15     45211 non-null  int64   
 15  V16     45211 non-null  category
 16  Class   45211 non-null  category
dtypes: category(10), int64(7)
memory usage: 2.8 MB


In [22]:
cat_cols=df.select_dtypes(include=['object','category']).columns
cat_cols

Index(['V2', 'V3', 'V4', 'V5', 'V7', 'V8', 'V9', 'V11', 'V16', 'Class'], dtype='object')

In [23]:
label={}
for i in cat_cols:
  le=LabelEncoder()
  df[i]=le.fit_transform(df[i])
  label[i]=le

In [24]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Class
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [27]:
X=df.drop(columns=['Class'])
y=df.Class

In [28]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)


If you standardize before splitting, the mean and standard deviation are computed from the entire dataset, including the test set. This causes data leakage, where information from the test set unintentionally influences the training process. Instead, standardizing after splitting ensures that only the training data is used to compute the mean and standard deviation.

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Compute mean/std and transform training data
X_test = scaler.transform(X_test)  # Use training mean/std to transform test data


In [30]:
model=keras.Sequential(
    [
        layers.Dense(32,activation='relu',input_shape=(X_train.shape[1],)), #input layer
        layers.Dense(16,activation='relu'), #hidden layer
        layers.Dense(8,activation='relu'),
        layers.Dense(1,activation='sigmoid') #output layer
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [36]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,217 (4.75 KB)

 Trainable params: 1,217 (4.75 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8617 - loss: 0.3257 - val_accuracy: 0.8904 - val_loss: 0.2510
Epoch 2/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8973 - loss: 0.2363 - val_accuracy: 0.8899 - val_loss: 0.2457
Epoch 3/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8993 - loss: 0.2342 - val_accuracy: 0.8947 - val_loss: 0.2408
Epoch 4/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9005 - loss: 0.2321 - val_accuracy: 0.8964 - val_loss: 0.2391
Epoch 5/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9011 - loss: 0.2262 - val_accuracy: 0.8985 - val_loss: 0.2398
Epoch 6/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9042 - loss: 0.2232 - val_accuracy: 0.8959 - val_loss: 0.2375
Epoch 7/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9044 - loss: 0.2194 - val_accuracy: 0.8977 - val_loss: 0.2365
Epoch 8/10
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9028 - loss: 0.2213 - 

In [38]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8978 - loss: 0.2351
Test Loss: 0.2346
Test Accuracy: 89.96%


In [45]:
new=np.random.rand(1,X_train.shape[1])
new

array([[0.25636935, 0.53613745, 0.5259251 , 0.05395954, 0.66790682,
        0.69563646, 0.27344727, 0.45638644, 0.63943766, 0.30131387,
        0.47683613, 0.77963253, 0.48161223, 0.40849753, 0.76917866,
        0.85787202]])

In [46]:
#for random values
prediction=model.predict(new)
prediction
predicted_label = 1 if prediction[0][0] > 0.5 else 0  # Convert probability to class
print(f"Predicted Probability: {prediction[0][0]:.4f}")
print(f"Predicted Label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Probability: 0.5515
Predicted Label: 1


In [42]:
# for real existing data
real_sample = X_test[:1]  # Take the first test example

# Get the true label for comparison
true_label = y_test.iloc[0]  # Use .iloc because y_test is a Pandas Series

# Predict the probability
prediction = model.predict(real_sample)
predicted_label = 1 if prediction[0][0] > 0.5 else 0  # Convert probability to class

print(f"Predicted Probability: {prediction[0][0]:.4f}")
print(f"Predicted Label: {predicted_label}, Actual Label: {true_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Probability: 0.0025
Predicted Label: 0, Actual Label: 0
